## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**4

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 47.2 ms ± 5.74 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24977.637632298945

Time taken by reduction operation using numpy.sum(): 95.1 µs ± 159 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 24977.637632298854 
 
Time taken by reduction operation using numpy.ndarray.sum(): 76.1 µs ± 180 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 24977.637632298854


In [4]:
value=5*10**7

 a) Librería multiprocessing: 

In [6]:
import numpy as np
import multiprocessing
import time
import sys

# Obtener el valor de 'value' desde los argumentos de línea de comandos
value = int(sys.argv[1])  # Este valor se pasa desde el script de Slurm

# Función poco optimizada para sumar elementos del array
def reduc_operation(A):
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Crear el array con el tamaño especificado en 'value'
X = np.random.rand(value)

# Función para dividir el array en N partes y usar multiprocessing
def reduc_operation_multiprocessing(n_parts, X):
    # Dividir el array en n_parts partes
    chunk_size = len(X) // n_parts
    chunks = [X[i:i + chunk_size] for i in range(0, len(X), chunk_size)]
    
    # Usar multiprocessing Pool para realizar la operación
    with multiprocessing.Pool(processes=n_parts) as pool:
        results = pool.map(reduc_operation, chunks)
    
    # Sumar los resultados de todos los procesos
    return sum(results)

# Medir el tiempo de ejecución para diferentes números de procesos
for n_parts in [1, 2, 4]:
    start_time = time.time()
    sum_result = reduc_operation_multiprocessing(n_parts, X)
    end_time = time.time()
    print(f"Tiempo con {n_parts} procesos: {end_time - start_time:.4f} segundos")


Tiempo con 1 procesos: 47.9190 segundos
Tiempo con 2 procesos: 25.6904 segundos
Tiempo con 4 procesos: 13.3741 segundos


 b) Libreria Numba:

In [7]:
import numba
from numba import njit, prange
import numpy as np
import sys
import time

value = int(sys.argv[1])

X = np.random.rand(value)

@njit
def reduc_operation_njit(A):
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

@njit(parallel=True)
def reduc_operation_njit_parallel(A):
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s

start_time = time.time()
sum_njit = reduc_operation_njit(X)
end_time = time.time()
# Medir el tiempo de ejecución para la versión sin paralelización
print(f"Tiempo con @njit: {end_time - start_time:.4f} segundos")

start_time = time.time()
sum_njit_parallel = reduc_operation_njit_parallel(X)
end_time = time.time()
# Medir el tiempo de ejecución para la versión con paralelización
print(f"Tiempo con @njit(parallel=True): {end_time - start_time:.4f} segundos")




Tiempo con @njit: 4.1207 segundos
Tiempo con @njit(parallel=True): 3.0468 segundos


Los resultados que aprecen en el propio script de notebook es de una resolucion previa, donde se introdujo el value que se observa al inicio.

Comparación d)

Tiempo con 1 procesos: 48.4510 segundos
Tiempo con 2 procesos: 25.6659 segundos
Tiempo con 4 procesos: 13.3436 segundos

Tiempo con @njit: 3.6187 segundos
Tiempo con @njit(parallel=True): 3.1310 segundos

Multiprocessing y Numba ofrecen dos enfoques distintos para optimizar el rendimiento de tareas computacionales, y sus diferencias son notables en los resultados obtenidos.

Con multiprocessing, el tiempo de ejecución disminuye a medida que se incrementa el número de procesos, ya que la carga de trabajo se distribuye entre varios núcleos de la CPU. Sin embargo, la mejora no es completamente lineal debido a la sobrecarga generada por la gestión de procesos, como la sincronización y la transferencia de datos entre ellos. Esto puede causar cuellos de botella, especialmente si la comunicación entre procesos es intensa o si el hardware tiene limitaciones en la cantidad de núcleos disponibles.

Por otro lado, Numba utiliza un enfoque diferente mediante la compilación Just-In-Time (JIT), que traduce el código Python a código máquina optimizado en tiempo de ejecución. La optimización secuencial de Numba ya es significativamente más rápida que multiprocessing debido a esta conversión directa. Cuando se activa la opción parallel=True, Numba aprovecha el paralelismo utilizando hilos en lugar de procesos. Esto reduce la sobrecarga asociada a la gestión de procesos independientes, ya que los hilos comparten el mismo espacio de memoria y no requieren la copia de datos.

En resumen, mientras que multiprocessing es adecuado para dividir tareas entre múltiples núcleos, su eficiencia se ve limitada por la sobrecarga de procesos. Por su parte, Numba sobresale en tareas numéricas intensivas gracias a su capacidad de compilación y manejo eficiente del paralelismo con hilos, lo que explica su desempeño superior.